Project 3 Fake News Detection

In [4]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import  PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [3]:
# Loading Dataset
df = pd.read_csv('/content/train.csv')

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df.tail()

,id,title,author,text,label
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [9]:
df.shape

(20800, 5)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [11]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [12]:
# replacing the null vlaues with empty string
df=df.fillna('')

In [13]:
# merging the author name and news title
df['content']= df['author']+''+df['title']

In [14]:
print(df['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [15]:
# seprating the data & label
X= df.drop(columns='label', axis=1)
Y=df['label']

In [16]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [17]:
port_stem =PorterStemmer()

In [20]:
# Define the stemming function
def stemming(content):
    content = content.lower()  # Convert to lowercase
    content = content.split()  # Split into words
    content = [port_stem.stem(word) for word in content if word not in stopwords.words('english')]  # Apply stemming
    return ' '.join(content)

In [21]:
df['content'] = df['content'].apply(stemming)

In [22]:
print(df['content'])

0        darrel lucushous dem aide: didn’t even see com...
1        daniel j. flynnflynn: hillari clinton, big wom...
2               consortiumnews.comwhi truth might get fire
3        jessica purkiss15 civilian kill singl us airst...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp t.i.: trump ’poster child whi...
20796    benjamin hoffmann.f.l. playoffs: schedule, mat...
20797    michael j. de la merc rachel abramsmacy’ said ...
20798    alex ansarynato, russia hold parallel exercis ...
20799                     david swansonwhat keep f-35 aliv
Name: content, Length: 20800, dtype: object


In [23]:
X=df['content'].values
Y=df['label']

In [24]:
print(X)

['darrel lucushous dem aide: didn’t even see comey’ letter jason chaffetz tweet'
 'daniel j. flynnflynn: hillari clinton, big woman campu - breitbart'
 'consortiumnews.comwhi truth might get fire' ...
 'michael j. de la merc rachel abramsmacy’ said receiv takeov approach hudson’ bay - new york time'
 'alex ansarynato, russia hold parallel exercis balkan'
 'david swansonwhat keep f-35 aliv']


In [25]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


In [26]:
Y.shape

(20800,)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)

In [32]:
print(X)

  (0, 1090)	0.36074580999296135
  (0, 5117)	0.3339547020931297
  (0, 5991)	0.22776883917952795
  (0, 7127)	0.3269077189450684
  (0, 7469)	0.26535031144709154
  (0, 7770)	0.2725285388588018
  (0, 9503)	0.2131010345817379
  (0, 14864)	0.2267902704938032
  (0, 16772)	0.2687730240968852
  (0, 17287)	0.39616172821890755
  (0, 25953)	0.23633175774140575
  (0, 30011)	0.2702317405535354
  (1, 3346)	0.35413476846519715
  (1, 4070)	0.18539331285820687
  (1, 4720)	0.464055696418436
  (1, 5725)	0.23248708292981804
  (1, 7068)	0.31638169487081524
  (1, 10395)	0.5226747357728401
  (1, 13172)	0.2508638367430399
  (1, 31859)	0.36688013213305515
  (2, 6139)	0.5493555185681402
  (2, 6291)	0.404084133229652
  (2, 10169)	0.3472323577842834
  (2, 11172)	0.3064339824029749
  (2, 18743)	0.4315491786494108
  :	:
  (20797, 2946)	0.33458674988742226
  (20797, 7255)	0.21214327969114827
  (20797, 13600)	0.30397444381925093
  (20797, 16265)	0.2268103670940579
  (20797, 18589)	0.32616316601624606
  (20797, 18695)	0

In [35]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [38]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(16640, 33438)
(4160, 33438)
(16640,)
(4160,)


In [39]:
# traning the logistic Regression Model
model= LogisticRegression()

In [40]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

Accuracy Score

In [42]:
# Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [43]:
print("The accuracy score of train data is", training_data_accuracy)

The accuracy score of train data is 0.9715745192307692


In [44]:
# Accuracy score on the testing data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [45]:
print("The accuracy score of train data is", testing_data_accuracy)

The accuracy score of train data is 0.9550480769230769


Making a Predictive system

In [48]:
X_new = X_test[0]

prediction= model.predict(X_new)

print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [50]:
print(Y_test[0])

1
